In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
import pandas as pd
import re

In [3]:
import gensim
from gensim.models import Word2Vec

In [3]:
pd.options.display.max_columns = None

In [4]:
df3 = pd.read_csv('data_processed_add_stems.csv', encoding = 'utf-16')

In [9]:
resumes2 = pd.read_csv('resumes_step2_stems.csv', encoding = 'utf-16')

In [12]:
resumes2.head(3)

,Unnamed: 0,birth_date,gender,area,education,title,education_level,salary.amount,salary.currency,specialization,skill_set,skills_desc,exp_length,exp_description,skills_desc_stem,exp_desc_stem
0,0,27 октября 1954,Мужчина,Москва,"[{'year': 1976, 'name': 'Высшее Военное Инжене...","Организатор корпоративных охот, рыбалок и эко-...",Высшее образование,60000.0,руб.,Руководитель проектов,"Работа в Фондах, НКО и ТПП, Союзах и Ассоциаци...",В работе: Аналитик.Трудоголик. Хобби: 50 лет ...,10915.0,Руководство административно-хозяйственной деят...,в работ аналитик трудоголик хобб лет занима ох...,руководств административн хозяйствен деятельн ...
1,1,7 июля 1978,Мужчина,Москва,"[{'year': 2002, 'name': 'Таганрогский Радиотех...",Руководитель проектов,Высшее образование,NaN,NaN,Руководитель проектов,Работоспособность,"Являюсь разносторонним человеком, люблю хорошо...",7554.0,"Организация рабочего процесса на объекте , вза...",явля разносторон человек любл хорош и плодотво...,организац рабоч процесс на объект взаимодейств...
2,2,20 января 1986,Мужчина,Москва,"[{'year': 2012, 'name': 'Институт Проблем Риск...",Руководитель проекта,Высшее образование,NaN,NaN,Руководитель проектов,"Автоматизация процессов,Проектное управление,О...","Системное мышление, глубокая аналитика и деком...",3732.0,Формирование бизнес требований. Сбор информац...,системн мышлен глубок аналитик и декомпозиц пр...,формирован бизнес требован сбор информац описа...


In [11]:
df3.head(3)

,Unnamed: 0,id,name,salary.from,salary.to,alternate_url,salary.currency,salary.gross,experience.name,experience.id,schedule.name,employment.name,employer.name,requirement,skills,description,prof_role,specializations,description_stem
0,0,67716839,Специалист по работе с покупателями,50000.0,NaN,https://hh.ru/vacancy/67716839,RUR,False,От 1 года до 3 лет,between1And3,Полный день,Полная занятость,ВкусВилл,ПК или ноутбук. Устойчивый интернет.,"Пользователь ПК,Грамотная речь,Поиск информаци...",Наша поддержка с душой реагирует на каждое обр...,Специалист технической поддержки,"Информационные технологии, интернет, телеком",наш поддержк с душ реагир на кажд обращен и вс...
1,1,68539555,Frontend разработчик (junior/middle),60000.0,120000.0,https://hh.ru/vacancy/68539555,RUR,False,От 1 года до 3 лет,between1And3,Полный день,Полная занятость,Инфоурок,"HTML(5)/CSS(3), Flex, кроссбраузерность. Препр...","JavaScript,Git,jQuery,HTML5,Sass",О нас: Продуктовая компания (основной продукт...,"Программист, разработчик","Информационные технологии, интернет, телеком",о нас продуктов компан основн продукт сайт inf...
2,2,68731055,Корректор контента,80000.0,NaN,https://hh.ru/vacancy/68731055,RUR,True,От 1 года до 3 лет,between1And3,Удаленная работа,Полная занятость,Российская академия народного хозяйства и госу...,"Знание Microsoft Office (PowerPoint, Word, Exc...","Грамотность,Грамотная речь,Работа в команде,Де...",В команду по развитию онлайн-образования требу...,"Копирайтер, редактор, корректор","Искусство, развлечения, масс-медиа;Информацион...",в команд по развит онлайн образован треб специ...


# Обучение модели

In [11]:
w2v_model = Word2Vec(min_count=1, window=2, vector_size=300, negative=10, alpha=0.03, min_alpha=0.0007, sample=6e-5, sg=1)

In [12]:
sentences = list(df3['description_stem'].fillna('--'))
new_sentences = [list(i.split(' ')) for i in sentences] 
sentences = new_sentences

In [13]:
%%time

w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=5, report_delay=1)

Wall time: 2min 1s


(10304233, 22604725)

In [15]:
w2v_model.save("w2v_description.model")

In [13]:
w2v_model = Word2Vec.load("w2v_description.model")

# Применение модели

In [14]:
def clear_txt(txt):
    stemmer = SnowballStemmer('russian')
    txt = txt.lower()
    txt = re.sub('[/+_!@#$0-9\n.,:()""«»;-]', ' ', txt)
    new_txt = ''
    for t in txt.split(' '):
        if len(t) > 0:
            new_txt = new_txt + stemmer.stem(t) + ' '
    return new_txt[:-1]

In [15]:

def match_description(df3, exp_desc):
    
    w2v_model = Word2Vec.load("w2v_description.model")
    
    df3['description_stem'] = df3['description_stem'].fillna('--')
    
    sentences = list(df3['description_stem'])
    new_sentences = [list(i.split(' ')) for i in sentences] 
    sentences = new_sentences

    target_sentence = exp_desc.replace(',', ' ').lower()
    target_sentence

    
    sentences_similarity = np.zeros(len(sentences))
    indexes = np.zeros(len(sentences))

    target_sentence_words = [w for w in target_sentence.split(' ') if w in w2v_model.wv.index_to_key]

    for idx, sentence in enumerate(sentences):
        
        sentence_words = [w for w in sentence if w in w2v_model.wv.index_to_key]

        sim = w2v_model.wv.n_similarity(target_sentence_words, sentence_words)
        sentences_similarity[idx] = sim
        
        ss = ' '.join(sentence)
        indexes[idx] = df3[df3['description_stem']==ss].index[0]

    #result = list(zip(indexes, sentences_similarity, sentences))
    result = list(zip(indexes, sentences_similarity))
    result.sort(key=lambda item:item[1], reverse=True)
    print("Target:", target_sentence)
    
    df = pd.DataFrame(zip(indexes, sentences_similarity), columns=['Indexes', 'Desc_Similarity'])
    
    ids = []

    for x in result[:5]:
        ids.append (x[0])
    
    return (ids)

# Пример поиска вакансий по описанию опыта

In [16]:
# выберем скиллы из датасета резюме

target_sentence = resumes2.loc[10, 'exp_description'].replace(',', ' ').lower()
target_sentence

'переговоры с внутренними заказчиками  выявление у них требований для модификации или разработки отчётности. работа с хранилищем данных  субд oracle. написание скрипта посредством sql для выгрузки этих данных. написание процедуры на языке pl/sql (посредством изменения заранее подготовленного шаблона) для автоматизации выгрузок и отсылки по корпоративной почте готовой отчетности заказчику.работа в компании  внедряющей решения на платформе "1с:предприятие 8.3" для страховых компаний. выполняемые мной задачи : меппинг таксономии  доработка конфигураций  разработка внешних печатных форм  исправление ошибок  обновление типовых и нетиповых конфигураций 1с.'

In [17]:
# на всех вакансиях

print (datetime.now())

ids = match_description(df3, 'переговор с внутрен заказчик выявлен у них требован для модификац ил разработк \
отч тност работ с хранилищ дан субд oracle написан скрипт посредств sql для выгрузк эт дан написан процедур \
на язык pl sql посредств изменен заран подготовлен шаблон для автоматизац выгрузок и отсылк по корпоративн \
почт готов отчетн заказчик работ в компан внедря решен на платформ с предприят для страхов компан выполня \
мно задач меппинг таксоном доработк конфигурац разработк внешн печатн форм исправлен ошибок обновлен типов \
и нетипов конфигурац с')

print (datetime.now())

2022-08-30 00:06:45.297434
Target: переговор с внутрен заказчик выявлен у них требован для модификац ил разработк отч тност работ с хранилищ дан субд oracle написан скрипт посредств sql для выгрузк эт дан написан процедур на язык pl sql посредств изменен заран подготовлен шаблон для автоматизац выгрузок и отсылк по корпоративн почт готов отчетн заказчик работ в компан внедря решен на платформ с предприят для страхов компан выполня мно задач меппинг таксоном доработк конфигурац разработк внешн печатн форм исправлен ошибок обновлен типов и нетипов конфигурац с
2022-08-30 00:13:45.976001


In [18]:
# requirement - "краткое описание" требуемого опыта, preview вакансии
# сам матчинг проходит по полному описанию

df3[df3.index.isin(ids) == True][['name', 'alternate_url', 'requirement']].style.hide_index()

name,alternate_url,requirement
Аналитик данных (Отчетность РБ),https://hh.ru/vacancy/66696168,Какие знания и навыки для нас важны: Уверенные знания SQL и написание сложных запросов / процедур. Опыт работы с реляционными ХД...
Аналитик данных,https://hh.ru/vacancy/67158351,Опыт работы в роди аналитика данных или системного аналитика в банке или страховой компании. Навыки программирования на SQL в части...
Бизнес-аналитик,https://hh.ru/vacancy/67634801,Опыт работы Бизнес - аналитиком.
Системный аналитик,https://hh.ru/vacancy/68146282,"Высшее образование в одной из областей: информационные технологии, финансы, экономика. Опыт работы системным аналитиком от 1 года (выявление, фиксация, согласование..."
Аналитик данных,https://hh.ru/vacancy/68792579,Опыт участия в проектах по созданию и развитию DWH от 2-х лет. Опыт участия в разработке витрин данных.
